<br>
<center><h1>  Categorical Decision Tree Creation Algorithm Report </h1></center>

***  

**&ensp;Authors:** Cameron Wright & Dylan Hammond  
**&ensp;&ensp;Course:** Artificial Intelligence [CSE 4301]  

**Professor:** Dr. Debasis Mitra  
**&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;TA:** Valerie Kobzarenko  

**&ensp;&ensp;&ensp;&ensp;&ensp;Date:** November 17th, 2021  

<br>

<br>

<a name="Table_of_contents"></a>
<center><h1> Table of contents </h1></center>
    
***  


- [**Table of contents**](#Table_of_contents)  
- [**Source Code**](#Source_Code)  
- [**Examples**](#Examples)  
- [**Report**](#Report)  
- [**References**](#References)  



<br>

**Implementation**

In [1]:
%load_ext watermark
%watermark -v -iv


Python implementation: CPython
Python version       : 3.9.1
IPython version      : 7.29.0



<br>
<a name="Source_Code"></a>
<center><h2> Source Code </h2></center>

*** 


<br>

**External Libraries**

In [2]:
# Arrays 
import numpy as np
# Tables
import pandas as pd
# Table display & style
from IPython.display import HTML
# Pesudo infinity value for info gain
from math import inf
# Display decision tree dictonary
from pprint import pprint
# Confusion matrix
from sklearn.metrics import confusion_matrix


<br>

**Table Helper Class**

In [3]:
# Discretely store headers (column names) and data (table body) of table
class tableLike:
    def __init__(self, headers, data):
        # Headers of the given table (always the first row)
        self.headers = headers
        # Body of the given table (all rows below the headers)
        self.data = data


<br>

**Purity Check**

In [4]:
# Check if target (label) values are pure (all the same categorical element)
def pure(tableData):
    # Target (label) values, always the last column of table [array]
    target = np.unique(tableData[:, -1])
    
    # If target (label) as only one type of unique element, it is pure [True]
    if len(target) == 1:
        return True
    # More than one unique element, it is NOT pure [False]
    return False


<br>

**Classification of Labels**

In [5]:
def classifyTable(tableData):
    # Target column
    target = tableData[:, -1]
    # Unique classes in target column [array] & number of each [array]
    classes, classCount = np.unique(target, return_counts=True)
    # Class that appears most in target column [string]
    bestClass = classes[classCount.argmax()]
    
    return bestClass


<br>

**Find All Splits**

In [6]:
# Return all possible splits [dictionary] for the given table data
def getSplits(tableData):
    # Dictionary to store all possible splits found
    validSplits = {}
    
    # Iterate through every column in table data except target
    for index, column in enumerate(tableData[:,:-1].T):
        # Store all unique elements from current column (classes)
        clss = np.unique(column)
        validSplits[index] = clss
        
    return validSplits


<br>

**Entropy Calculation**

In [7]:
# Return entropy [float] (a measure of randomness) for the given table data 
def entropy(tableData):
    # All unqiue elements
    classes = np.unique(tableData)
    # Calculate entropy for all classes
    entropy = 0
    for clss in classes:
        # Probability of current class 
        probClss = len(tableData[tableData == clss]) / len(tableData)
        # Entropy of current class
        entropy += -probClss * np.log2(probClss)
    
    # Total entropy over all classes
    return entropy


<br>

**Information Gain**

In [8]:
# Return information gain [float] for the given split
def infoGain(parent, leftChild, rightChild):
    # Probability of left child
    leftWeight = len(leftChild) / len(parent)
    # Probability of right child
    rightWeight = len(rightChild) / len(parent)
    # Entropy of parent - average weighted entropy of children
    gain = entropy(parent) - (leftWeight*entropy(leftChild) + rightWeight*entropy(rightChild))
    
    # Information gain for the given split [float]
    return gain


<br>

**Split Table**

In [9]:
# Return split table data [arrays] based on equality to the given class (clss)
def split(tableData, index, clss):
    # Store data in the column, that will be used to determine split [array]
    splitColumn = tableData[:, index]
    # Split rows based on if they are the given class
    equalRows = tableData[splitColumn == clss]
    unequalRows = tableData[splitColumn != clss] 

    # Return the split row groups
    return equalRows, unequalRows


<br>

**Find Best Split**

In [10]:
# Return column index and class [dict] of the split with greatest information gain
def bestSplit(tableData):
    # Minimum information gain possiable
    minGain = -inf
    
    # Get all possiable splits
    validSplits = getSplits(tableData)
    # Go through every split
    for columnIndex in validSplits:
        for clss in validSplits[columnIndex]:
            # Get corresponding split rows
            equal, unequal = split(tableData, columnIndex, clss)
            # Info gain for target column on split
            currGain = infoGain(tableData[:, -1], equal[:, -1], unequal[:, -1])
            
            # Check if split yields more information gain than previous best
            if currGain > minGain:
                minGain = currGain
                bestIndex = columnIndex
                bestClss = clss
    
    # Best split found [dict]
    return {'index':bestIndex, 'value':bestClss}


<br>

**Build Decison Tree**

In [11]:
# Build decision tree of nested dictionaries for given table data and headers
def plant(data, headers, depth=0, minSplit=2, maxDepth=99):
    # Base cases (leaf node)
    if  len(data) < minSplit or depth ==  maxDepth or pure(data):
        # Classification of label column
        return classifyTable(data)
    # Continue, the current node has children
    else:
        # Get best splits of all potential splits
        node = bestSplit(data)
        left, right = split(data, node["index"], node["value"])
        
        # Split rows should NOT be empty
        if len(left) == 0 or len(right) == 0:
            return classifyTable(data)
        
        # Nested dictionary structure for subtree
        query = "%d | %s = %s" % (node['index'], headers[node['index']], node['value'])
        subTree = {query: []}
        
        # Continue on each child node
        left = plant(left, headers, depth+1, minSplit, maxDepth)
        right = plant(right, headers, depth+1, minSplit, maxDepth)
        
        # If children are identical should only be one
        if left == right:
            subTree = left
        else:
            # Append children
            subTree[query].append(left)
            subTree[query].append(right)
        return subTree
        

<br>

**Node Format**

      parent   left child, right child
    { query: [ true group, false group ] }

<br>

**Classify Tree**

In [12]:
# Return lables for given tree [array]
def classifyTree(tableData, tree):
    # Parse current query
    query = list(tree.keys())[0]
    # Column index, '|', Column header, '=', Label value
    colIndex, _, header, _, value = query.split()

    # Equal rows, continue on left subtree
    if tableData[int(colIndex)] == value:
        decision = tree[query][0]
    else:
        # Unequal rows, continue on right subtree
        decision = tree[query][1]

    # If decision is not a dictionary, base case reached (leaf node)
    if not isinstance(decision, dict):
        return decision
    else:
        # Recursively continue on remaining subtree
        return classifyTree(tableData, decision)


<br>

**Display Confusion Matrix**

In [13]:
# Display confusion matrix for labels of the actual table & those predicted by tree
def confusionMatrix(table, tree):
    # np.apply_along_axis does not like string dtype (will truncate)
    table = np.array(table, dtype=object)
    
    # Predicted table labels
    predicted = pd.Series([classifyTree(table[i], tree) for i in range(table.shape[0])], name='Actual')
    # Actual table labels
    actual = pd.Series(table[:,-1].tolist(), name='Predicted')
    
    # Display confusion matrix
    print(pd.crosstab(actual, predicted, rownames=['Actual'], colnames=['Predicted'], margins=True))
    

<br>
<a name="Examples"></a>
<center><h2> Examples </h2></center>

*** 


<br>

**Local directory contents (Example files)**

In [14]:
# For accessing & displaying contents (files) of the working (local) directory
from os import listdir
from os.path import isfile
from os.path import join
from os.path import abspath

# Path to current working directory
dir_path = abspath('')
#dir_path = "D:\Documents\Jupyter_Notebooks\AI"

# Store example files [.txt] in the local directory [list]
files = [f for f in listdir(dir_path)
         if isfile(join(dir_path, f)) and f[-3:] == 'txt']

# Output all local example files
print('\n'.join(files))


mammal.txt
restaurant.txt
tennis.txt
vertebrate.txt


<br>

**Display & Return Table**

In [15]:
# Display dataframe [HTML] & Return tableLike [class] given its path (i.e. "~\folder\table.txt")
def GetTable(path):
    # Read in table ('\t' as seperator) from path as a pandas dataframe, first row set as headers
    table = pd.read_table(path, header=0)
    # Table header (column) names as 1d numpy array
    headers = table.columns
    # Store table body values as numpy array
    data = table.values

    # Display table [pandas dataframe] reterned as HTML table and stylized using CSS
    display(HTML('''
        <style>
            table.dataframe table {
                border-collapse: collapse;
            }
            table.dataframe td:first-child { border-left: none; }
            table.dataframe th {
                text-align: left;
                font-weight: bold;
            }
            table.dataframe td { 
                border-left: 0.75px solid lightgrey;
                border-bottom: 0.75px solid lightgrey;
                text-align: left;
            }
            table.dataframe thead th {
                background-color: #e8e8e8;
                border-bottom: 3px solid black; 
            }
            table.dataframe tbody th{ border-right: 2px solid black; }
            table.dataframe tr:nth-child(even){ background-color: #e0f1ff; }
            table.dataframe tbody th:nth-child(even) { background-color: #e0f1ff; }
              
        </style>
        ''' + table.to_html(classes="table")))
    
    # Return table as list of headers  (without index column) columns preserved with dtype (i.e. <arry>.dtype.name)
    return tableLike(headers, data)


<br>
<a name="mammal"></a>

**mammal**

In [16]:
table = GetTable("mammal.txt")

,Name,Body-Temperature,Gives-Birth,Four-legged,Hibernates,Class-Label
0,salamander,cold-blooded,no,yes,yes,no
1,guppy,cold-blooded,yes,no,no,no
2,eagle,warm-blooded,no,no,no,no
3,poorwill,warm-blooded,no,no,yes,no
4,platypus,warm-blooded,no,yes,yes,yes


*Create decision tree:*

In [17]:
tree = plant(table.data, table.headers)
pprint(tree,indent=0)

{'0 | Name = platypus': ['yes', 'no']}


*Corresponding confusion matrix:*

In [18]:
confusionMatrix(table.data, tree)

Predicted  no  yes  All
Actual                 
no          4    0    4
yes         0    1    1
All         4    1    5


<br>
<a name="tennis"></a>

**tennis**

In [19]:
table = GetTable("tennis.txt")

,Outlook,Temperature,Humidity,Wind,PlayTennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rain,Mild,Normal,Weak,Yes


*Create & display decision tree:*

In [20]:
tree = plant(table.data, table.headers)
pprint(tree,indent=1)

{'0 | Outlook = Overcast': ['Yes',
                            {'2 | Humidity = High': [{'0 | Outlook = Rain': [{'3 | Wind = Strong': ['No',
                                                                                                    'Yes']},
                                                                             'No']},
                                                     {'3 | Wind = Strong': [{'0 | Outlook = Rain': ['No',
                                                                                                    'Yes']},
                                                                            'Yes']}]}]}


*Corresponding confusion matrix:*

In [21]:
confusionMatrix(table.data, tree)

Predicted  No  Yes  All
Actual                 
No          5    0    5
Yes         0    9    9
All         5    9   14


<br>
<a name="restaurant"></a>

**restaurant**

In [22]:
table = GetTable("restaurant.txt")

,Alt,Bar,Fri,Hun,Pat,Price,Rain,Res,Type,Est,WillWait
0,Yes,No,No,Yes,Some,$$$,No,Yes,French,0-10,Yes
1,Yes,No,No,Yes,Full,$,No,No,Thai,30-60,No
2,No,Yes,No,No,Some,$,No,No,Burger,0-10,Yes
3,Yes,No,Yes,Yes,Full,$,Yes,No,Thai,10-30,Yes
4,Yes,No,Yes,No,Full,$$$,No,Yes,French,>60,No
5,No,Yes,No,Yes,Some,$$,Yes,Yes,Italian,0-10,Yes
6,No,Yes,No,No,None,$,Yes,No,Burger,0-10,No
7,No,No,No,Yes,Some,$$,Yes,Yes,Thai,0-10,Yes
8,No,Yes,Yes,No,Full,$,Yes,No,Burger,>60,No
9,Yes,Yes,Yes,Yes,Full,$$$,No,Yes,Italian,10-30,No


*Create & display decision tree:*

In [23]:
tree = plant(table.data, table.headers)
pprint(tree,indent=1)

{'4 | Pat = Some': ['Yes',
                    {'3 | Hun = No': ['No',
                                      {'2 | Fri = No': ['No',
                                                        {'5 | Price = $': ['Yes',
                                                                           'No']}]}]}]}


*Corresponding confusion matrix:*

In [24]:
confusionMatrix(table.data, tree)

Predicted  No  Yes  All
Actual                 
No          6    0    6
Yes         0    6    6
All         6    6   12


<br>
<a name="vertebrate"></a>

**vertebrate**

In [25]:
table = GetTable("vertebrate.txt")

,Name,Body-Temperature,Skin-Cover,Gives-Birth,Aquatic-Creature,Aerial-Creature,Has-Legs,Hiber-nates,Class-Label
0,human,warm-blooded,hair,yes,no,no,yes,no,mammal
1,python,cold-blooded,scales,no,no,no,no,yes,reptile
2,salmon,cold-blooded,scales,no,yes,no,no,no,fish
3,whale,warm-blooded,hair,yes,yes,no,no,no,mammal
4,frog,cold-blooded,none,no,semi,no,yes,yes,amphibian
5,komodo,cold-blooded,scales,no,no,no,yes,no,reptile
6,bat,warm-blooded,hair,yes,no,yes,yes,yes,mammal
7,pigeon,warm-blooded,feathers,no,no,yes,yes,no,bird
8,cat,warm-blooded,fur,yes,no,no,yes,no,mammal
9,leopard,cold-blooded,scales,yes,yes,no,no,no,fish


*Create & display decision tree:*

In [26]:
tree = plant(table.data, table.headers)
pprint(tree,indent=1)

{'1 | Body-Temperature = cold-blooded': [{'4 | Aquatic-Creature = yes': ['fish',
                                                                         {'2 | Skin-Cover = none': ['amphibian',
                                                                                                    'reptile']}]},
                                         {'2 | Skin-Cover = feathers': ['bird',
                                                                        'mammal']}]}


*Corresponding confusion matrix:*

In [27]:
confusionMatrix(table.data, tree)

Predicted  amphibian  bird  fish  mammal  reptile  All
Actual                                                
amphibian          2     0     0       0        0    2
bird               0     2     0       0        0    2
fish               0     0     3       0        0    3
mammal             0     0     0       5        0    5
reptile            0     0     0       0        3    3
All                2     2     3       5        3   15


<br>
<a name="Report"></a> 
<center><h2> Report </h2></center> 
<br>


**Overview**

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;The above implementation is a function-centric implementation of a categorical decision tree creation algorithm. This is due to my familiarity with this paradigm of programming. Though traditionally, a more class-centric approach is taken, and admittedly such an implementation is more desirable. However, a function-centric approach enables the use of cell-oriented programming in a notebook such as this. Moreover, I opted to segment the functionality as much as possible to clarify what was occurring throughout the program's source code. This allowed me to incrementally check my work and made it easier to understand for those viewing it. 

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;I chose sources similar to the original Restaurant example given in figure 18.2 of the textbook. This allows for the entire tree to be displayed, and it could be easily validated. However, admittedly running on such small tables is impractical.  

<br>

**Specifics**

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;The example input files are simple text files with the traditional layout of a table (.csv) file, except the separators are expected to be tab characters as specified in the prompt for this project. The example input is taken in as a pandas dataframe; this allows each table to be displayed nicely using HTML formatting. However, after displaying, the dataframe is turned into a NumPy array, and its column names and body are separated and stored in a class. The table remains as this tableLike class containing NumPy arrays until the confusion matrix is displayed; at that point, it is turned into a pandas series to be displayed using the sklearn confusion_matrix function.  
<br>


**Improvements**

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Implementing a decision tree from such a low level is regarding, but they're already exists well-defined libraries that can do it better, such as sklearn. I would have preferred to use external functions in place of my own while maintaining the level of comments in the current implementation. Making the implementation faster while maintaining clarity.  

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;The use of NumPy arrays as a method of storing categorical tables is less than ideal, as it is not designed with the string data type in mind. As a result, speed's primary benefit of using a NumPy array is negated when working with strings. Moreover, I ran into many know issues with NumPy functions that don't work with strings. Because of this, I would opt to keep the table as a pandas dataframe in future implementations.  
<br>



<br>
<a name="References"></a>
<center><h2> References </h2></center> 
<br>


**Concepts**  
*The following gave a further overview of Decision Trees & how to build them*

This [Decision Tree Outline Document](https://www.cs.cmu.edu/~bhiksha/courses/10-601/decisiontrees/) by Dr.Raj on Carnegie Mellon University's website.  

The textbook for this course, [Artificial intelligence: A modern approach](https://cs.calvin.edu/courses/cs/344/kvlinden/resources/AIMA-3rd-edition.pdf) by Russell S & Norvig P.  
<br>

**Implementation**  
*The following gave specific code/function which was adapted*  

Mantey S's [Decision-Tree-from-Scratch](https://github.com/SebastianMantey/Decision-Tree-from-Scratch) GitHub repository  
<br>

**Examples**   
*All example tables were adapted from their following respective sources.*

Mammal & Vertebrate:  
Chapter 4 in [Introduction to Data Mining](https://www-users.cse.umn.edu/~kumar001/dmbook/index.php) by Tan P,  Steinbach M, et al.

Tennis:  
Chapter 3 in [Machine Learning](https://www.cin.ufpe.br/~cavmj/Machine%20-%20Learning%20-%20Tom%20Mitchell.pdf) by Mitchell T.  

Restaurant:  
Chapter 18 in [Artificial intelligence: A modern approach](https://cs.calvin.edu/courses/cs/344/kvlinden/resources/AIMA-3rd-edition.pdf) by Russell S & Norvig P.  
<br>

